In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import geohash_hilbert as ghh

In [2]:
#downloaded from TLC by month
df = pd.read_csv('data/fhv_tripdata_2015-01.csv')
print ("Shape of data\n{}".format(df.shape))
df.head()

Shape of data
(2746033, 3)


,Dispatching_base_num,Pickup_date,locationID
0,B00013,2015-01-01 00:30:00,NaN
1,B00013,2015-01-01 01:22:00,NaN
2,B00013,2015-01-01 01:23:00,NaN
3,B00013,2015-01-01 01:44:00,NaN
4,B00013,2015-01-01 02:00:00,NaN


In [3]:
df.columns

Index(['Dispatching_base_num', 'Pickup_date', 'locationID'], dtype='object')

In [4]:
df = df.drop(['locationID'], axis=1)
df.dropna(inplace=True)
df.head()

,Dispatching_base_num,Pickup_date
0,B00013,2015-01-01 00:30:00
1,B00013,2015-01-01 01:22:00
2,B00013,2015-01-01 01:23:00
3,B00013,2015-01-01 01:44:00
4,B00013,2015-01-01 02:00:00


In [5]:
df.shape

(2746033, 2)

## Aggregate by hour

In [6]:
df['Pickup_date'] = pd.to_datetime(df['Pickup_date'], format='%Y/%m/%d %H:%M:%S')

In [7]:
df.dtypes

Dispatching_base_num            object
Pickup_date             datetime64[ns]
dtype: object

In [8]:
df['Dispatching_base_num'].unique()

array(['B00013', 'B00014', 'B00053', 'B00095', 'B00225', 'B00227',
       'B00248', 'B00254', 'B00272', 'B00280', 'B00310', 'B00381',
       'B00412', 'B00448', 'B00457', 'B00477', 'B00628', 'B00837',
       'B00887', 'B01066', 'B01083', 'B01087', 'B01197', 'B01282',
       'B01285', 'B01300', 'B01312', 'B01313', 'B01315', 'B01341',
       'b01346', 'B01367    ', 'B01525', 'B01593', 'B01651', 'B01667',
       'B01777', 'B01811', 'B01848', 'B01877', 'B01899', 'B01984',
       'B02003', 'B02067', 'B02107', 'B02120', 'B02285', 'B02512',
       'B02598', 'B02617', 'B02653', 'B02682', 'B02715', 'B02764', 'B02765'], dtype=object)

In [9]:
df['hour'] = df['Pickup_date'].dt.hour
df['dayofyear'] = df['Pickup_date'].dt.dayofyear

In [10]:
df['day_and_hour'] = df['dayofyear'].astype(str) + df['hour'].astype(str)
df['FHV_count'] = pd.Series(np.ones(len(df)).T)

In [11]:
df.head()

,Dispatching_base_num,Pickup_date,hour,dayofyear,day_and_hour,FHV_count
0,B00013,2015-01-01 00:30:00,0,1,10,1.0
1,B00013,2015-01-01 01:22:00,1,1,11,1.0
2,B00013,2015-01-01 01:23:00,1,1,11,1.0
3,B00013,2015-01-01 01:44:00,1,1,11,1.0
4,B00013,2015-01-01 02:00:00,2,1,12,1.0


In [12]:
df = df.drop(['Dispatching_base_num'], axis=1)
df.dropna(inplace=True)
df.head()

,Pickup_date,hour,dayofyear,day_and_hour,FHV_count
0,2015-01-01 00:30:00,0,1,10,1.0
1,2015-01-01 01:22:00,1,1,11,1.0
2,2015-01-01 01:23:00,1,1,11,1.0
3,2015-01-01 01:44:00,1,1,11,1.0
4,2015-01-01 02:00:00,2,1,12,1.0


In [13]:
dfsum = df.groupby(df['day_and_hour']).count()
dfsum.head()

,Pickup_date,hour,dayofyear,FHV_count
day_and_hour,,,,
10,7455,7455,7455,7455
100,5080,5080,5080,5080
101,3551,3551,3551,3551
1010,3342,3342,3342,3342
1011,3666,3666,3666,3666


In [14]:
dfsum.reset_index(inplace=True)
dfsum.head()

,day_and_hour,Pickup_date,hour,dayofyear,FHV_count
0,10,7455,7455,7455,7455
1,100,5080,5080,5080,5080
2,101,3551,3551,3551,3551
3,1010,3342,3342,3342,3342
4,1011,3666,3666,3666,3666


In [15]:
dfsum.drop(['Pickup_date', 'hour', 'dayofyear'], axis=1, inplace=True)
dfsum.shape

(706, 2)

In [16]:
dfsum.head()

,day_and_hour,FHV_count
0,10,7455
1,100,5080
2,101,3551
3,1010,3342
4,1011,3666


In [18]:
dfsum.to_csv('clean_FHV_data.csv')